In [1]:
import pandas as pd
import numpy as np

In [2]:
db = pd.read_csv("train.csv")
db = db.reindex(np.random.permutation(db.index))
comment = db['comment_text'].as_matrix()
label = db[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']].as_matrix()
# print (comments)
comments = []
labels = []

for ix in range(comment.shape[0]):
    if len(comment[ix])<=400:
        comments.append(comment[ix])
        labels.append(label[ix])

In [3]:
labels = np.asarray(labels)

In [4]:
print(len(comments))

75481


In [5]:
from stop_words import get_stop_words
stop_words = get_stop_words('english')
stop_words.append('')

for x in range(ord('b'), ord('z')+1):
    stop_words.append(chr(x))

In [6]:
print (stop_words)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

In [7]:
import string
punctuation_edit = string.punctuation.replace('\'','') +"0123456789"
print ((punctuation_edit))
print (comments[1])
outtab = "                                         "
trantab = str.maketrans(punctuation_edit, outtab)

!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789
waterborne parasites and aquatic predators 
This page has obviously been 'sabotaged' by someone who is not familiar and hostile with AAH. Numerous opinions stated as fact, and strawman arguments added.  As far as controversial subjects are concerned, Wikipedia is ruined really isn't it!


In [8]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [9]:
lemmatiser = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/nupur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
for i in range(len(comments)):
# for i in range(5):
    comments[i] = comments[i].lower().translate(trantab)
    l = []
    for word in comments[i].split():
        l.append(stemmer.stem(lemmatiser.lemmatize(word,pos="v")))
    comments[i] = " ".join(l)
#     result_words[i] = re.split(' |\n',result_words[i])
#     result_words[i] = [word for word in result_words[i] if word not in stop_words]
# print(lemmatiser.lemmatize("having",pos="v"))

In [11]:
count_vector = CountVectorizer(stop_words=stop_words)
tf = count_vector.fit_transform(comments).toarray()

In [12]:
# import numpy as np
# from sklearn.decomposition import PCA

# pca = PCA(n_components=400)
# tf = pca.fit(tf)

In [13]:
# print(count_vector.get_feature_names())
print(tf.shape)

(75481, 59173)


In [14]:
from sklearn import svm

In [15]:
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion)
    print(ratio)
    X_train = matrix[ratio:,:]
    X_test =  matrix[:ratio,:]
    Y_train = target[ratio:,:]
    Y_test =  target[:ratio,:]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = shuffle(tf, labels, 3)

print(Y_train.shape)

25160
(50321, 6)


In [38]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [39]:
model = Sequential()
model.add(Dense(4, activation='relu', input_dim = X_train.shape[1]))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 4)                 236696    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                50        
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 66        
Total params: 236,812
Trainable params: 236,812
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [41]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.myneural.h5py', 
                               verbose=1, save_best_only=True)
model.fit(X_train, Y_train, epochs=10, batch_size=32)


Epoch 1/10
50321/50321 [==============================] - 79s 2ms/step - loss: 0.3583 - acc: 0.9566
Epoch 2/10
50321/50321 [==============================] - 69s 1ms/step - loss: 0.3351 - acc: 0.9847
Epoch 3/10
50321/50321 [==============================] - 66s 1ms/step - loss: 0.3314 - acc: 0.9842
Epoch 4/10
50321/50321 [==============================] - 68s 1ms/step - loss: 0.3267 - acc: 0.9841
Epoch 5/10
50321/50321 [==============================] - 70s 1ms/step - loss: 0.3230 - acc: 0.9840
Epoch 6/10
50321/50321 [==============================] - 65s 1ms/step - loss: 0.3201 - acc: 0.9817
Epoch 7/10
50321/50321 [==============================] - 67s 1ms/step - loss: 0.3193 - acc: 0.9816
Epoch 8/10
50321/50321 [==============================] - 66s 1ms/step - loss: 0.3187 - acc: 0.9797
Epoch 9/10
50321/50321 [==============================] - 66s 1ms/step - loss: 0.3166 - acc: 0.9801
Epoch 10/10
50321/50321 [==============================] - 66s 1ms/step - loss: 0.3160 - acc: 0.9786

In [20]:
model.load_weights('saved_models/weights.best.myneural.h5py')

OSError: Unable to open file (Unable to open file: name = 'saved_models/weights.best.myneural.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [30]:
predict = model.predict(X_test)

In [31]:
predict = np.round(predict)

In [ ]:
# predict = np.asarray(np.transpose(predict))
# print(predict.shape)

In [32]:
from sklearn.metrics import hamming_loss
loss = hamming_loss(Y_test,predict)
print(loss*100)

13.413487016428194


In [33]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test,predict)
print(accuracy*100)

31.5143084261


In [ ]:
# from skmultilearn.problem_transform import ClassifierChain

# classifier = ClassifierChain(MultinomialNB())
# classifier.fit(X_train, Y_train)

In [34]:
from sklearn.metrics import log_loss
print(log_loss(Y_test,predict))

1.35070790198
